In [ ]:
#!pip install scikit-image

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import morphology, io,exposure
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import img_as_ubyte,measure
from skimage.morphology import dilation, opening, erosion
import skimage
from skimage import color
import os
from skimage.segmentation import slic, mark_boundaries

In [ ]:
temp = plt.imread('../data/raw/example_segmentation/ISIC_0001769_segmentation.png')
temp_resized = resize(temp, (768, 1024),anti_aliasing=True)
            
io.imshow(img_as_ubyte(temp_resized))

In [ ]:
#Resize all images to 768x1024 and output to interim
#for filename in os.listdir('../data/raw/example_segmentation'):
#    if 'DS' in filename: 
#        continue
#    temp = plt.imread('../data/raw/example_segmentation/'+filename)
#    temp_resized = resize(temp, (768, 1024),anti_aliasing=True)

#    new_filename = filename[:5]+"re_"+filename[5:]                 
#    io.imsave('../data/interim/resized_segments_ver_2/'+new_filename, img_as_ubyte(temp_resized), check_contrast = False)

In [ ]:
im = plt.imread('../data/raw/example_image/ISIC_0003462.jpg')
mask=plt.imread('../data/raw/example_segmentation/ISIC_0003462_segmentation.png')


In [ ]:
#Total size of the image
total = mask.shape[0] * mask.shape[1] 

#Size of mask only
area = np.sum(mask)

#As percentage
print(area/total*100, "%")
plt.imshow(mask, cmap='gray')

In [ ]:
#Structural element, that we will use as a "brush" on our mask. The parameter is "brush size"
struct_el = morphology.disk(2)
print(struct_el)

# Use this "brush" to erode the image - eat away at the borders

mask_eroded = morphology.binary_erosion(mask, struct_el)

# Show side by side (depending on brush size, you might not see a difference visually)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 3))
axes[0].imshow(mask, cmap='gray')
axes[1].imshow(mask_eroded, cmap='gray')
fig.tight_layout()

# Verify the new mask is smaller
new_area= np.sum(mask_eroded)

print(area)
print(new_area)

In [ ]:
# Subtract the two masks from each other to get the border/perimeter

image_perimeter = mask - mask_eroded

plt.imshow(image_perimeter, cmap='gray') #Depending on the size of the "brush" it might be difficult to see on the screen

#What is the length of this perimeter = how many 1s? 
print(np.sum(image_perimeter))

In [ ]:
images = os.listdir('../data/interim/resized_images')
segments = os.listdir('../data/interim/resized_segments')
#Greyscaling
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

fim = plt.imread('../data/interim/resized_images/'+images[3])
fim = exposure.rescale_intensity(fim)
p2, p98 = np.percentile(fim, (2, 98))
fim = exposure.rescale_intensity(fim, in_range=(p2, p98))
fim_segment = plt.imread('../data/interim/resized_segments/'+segments[2])
gray_1 = exposure.rescale_intensity(fim[:,:,1]) #green only
gray_2 = exposure.rescale_intensity(fim[:,:,2]) #blue only
gray_3 = exposure.rescale_intensity(rgb2gray(fim)) #all greyscaled

plt.imshow(fim)
#plt.imshow(gray_3)

# red test cases: 1, 15, 93, 23
# green test cases: 

colors = ("r", "g", "b")
ids = (0, 1, 2)

hist_data = {}
for c,id in zip(colors,ids):
    histogram, bin_edges = np.histogram(fim[:,:,id], bins = 256, range = (0,256))
    hist_data[c] = histogram
#print(hist_data)

dictionary = {}
for c in colors:
    dictionary[c] = np.max(hist_data[c])
print(dictionary)

In [ ]:
#gray2 = gray[0:1500,:]
#plt.imshow(gray2, cmap='gray')

#mask2 = mask[0:1500,:]
#plt.imshow(mask2)

In [ ]:
def q(im):
    hist_data = {}
    for c,id in zip(colors,ids):
        histogram, bin_edges = np.histogram(fim[:,:,id], bins = 256, range = (0,256))
        hist_data[c] = histogram
    #print(hist_data)
    
    dictionary['r'] = sum([hist_data['r'][i]*i for i in range(len(hist_data['r']))])
    dictionary['g'] = sum([hist_data['g'][i]*i for i in range(len(hist_data['g']))])
    dictionary['b'] = sum([hist_data['b'][i]*i for i in range(len(hist_data['b']))])
    
    
    #max_pixel_count = max(dictionary.values())
    # if there are two maxes???
    quantile = 0
    
    ### i'll set the quantiles later after i test with a couple images
    
    #if max_pixel_count == dictionary["r"]:
        # if b and g <<< r, then quantile = 0.5 fits much better --> test case 23
        ## quantile = 0.2 fits much better for test case 60 ??
        #quantile = 0.3
    #elif max_pixel_count == dictionary["g"]:
        #quantile = 0.1
    #else:
        #quantile = 0.1
        
   
    
    return dictionary

plt.bar(q(fim).keys(),q(fim).values(),color=["red","green",'blue']);

In [ ]:
#Increase contrast for easier mask detection
temp = plt.imread('../data/interim/resized_images/'+problems[0])
temp_seg = plt.imread('../data/interim/resized_segments/'+problems[0][:-4]+"_segmentation.png")

temp = exposure.rescale_intensity(temp) 
gray_1 = exposure.rescale_intensity(temp[:,:,1]) #green only
gray_2 = exposure.rescale_intensity(temp[:,:,2]) #blue only
gray_3 = exposure.rescale_intensity(rgb2gray(temp)) #all greyscaled
imlist = []
difflist = []
same_counter = 0

struct_el = morphology.disk(2) #brush to remove most hairs
index = 0
for quant in np.arange(0.15,1,0.05):
    
    #first mask with green only
    mymask_1 = gray_1 < np.quantile(gray_1,quant) 
    mymask_1 = opening(mymask_1, struct_el)
    #second mask with green only
    mymask_2 = gray_2 < np.quantile(gray_2,quant) 
    mymask_2 = opening(mymask_2, struct_el)
    #third mask with greyscaled image
    mymask_3 = gray_3 < np.quantile(gray_3,quant) 
    mymask_3 = opening(mymask_3, struct_el)
    #Add all masks and keep joint elements
    better_mask = mymask_1.astype('int32')+mymask_2.astype('int32')+mymask_3.astype('int32')
    fixed_mask = better_mask.copy() + temp_seg
    
    #make mask binary (or rather 0's and 1's) then make morphology corrections
    fixed_mask[fixed_mask < 4] = 0
    fixed_mask[fixed_mask == 4] = 1
    fixed_mask = morphology.closing(fixed_mask,morphology.disk(3))
    fixed_mask = morphology.dilation(fixed_mask,morphology.disk(2))
    fixed_mask = morphology.area_closing(fixed_mask,100000)
    fixed_mask = morphology.remove_small_objects(fixed_mask.astype("bool"),3000)

    #calculate difference between current mask and original segmentation
    diff = np.abs((np.sum(fixed_mask))/np.sum(temp_seg)) 
    
    #keep the fixed_mask if it is at least 5% different than the original segmentation
    #mask cannot take 100% of the image size (768x1024)
    if((diff > 1.05 or diff < 0.95) and np.sum(fixed_mask)!=786432) :
        imlist.append(fixed_mask)
        difflist.append(np.abs(1-diff))
        index+=1
        same_counter = 0
    else:
        same_counter += 1
    
    
    #break when best mask has been found
    if index >=2:
        if difflist[index-1] > difflist[index-2] or same_counter >= 3:
            print(quant)
            break
        
    
    



f, ax = plt.subplots(1,3,figsize=(30,10))
ax[0].imshow(temp,cmap='gray')
ax[1].imshow(temp_seg,cmap='gray')
ax[2].imshow(imlist[difflist.index(min(difflist))] if imlist else temp_seg,cmap='gray')

#ax[1,1].imshow(imlist[2],cmap='gray')
#ax[2,0].imshow(imlist[5],cmap='gray')
#ax[2,1].imshow(imlist[7],cmap='gray')

print(difflist.index(min(difflist)))
print(np.sum(imlist[difflist.index(min(difflist))]))
print(difflist)

In [ ]:
#Create custom masks for all images (without using preexisting masks)

for filename in os.listdir('../data/interim/resized_images'):
    temp = plt.imread('../data/interim/resized_images/'+filename)
    temp_seg = plt.imread('../data/interim/resized_segments/'+filename[:-4]+"_segmentation.png")

    temp = exposure.rescale_intensity(temp) 
    gray_1 = exposure.rescale_intensity(temp[:,:,1]) #green only
    gray_2 = exposure.rescale_intensity(temp[:,:,2]) #blue only
    gray_3 = exposure.rescale_intensity(rgb2gray(temp)) #all greyscaled
    imlist = []
    difflist = []
    same_counter = 0

    struct_el = morphology.disk(2) #brush to remove most hairs
    index = 0
    for quant in np.arange(0.15,1,0.05):

        #first mask with green only
        mymask_1 = gray_1 < np.quantile(gray_1,quant) 
        mymask_1 = opening(mymask_1, struct_el)
        #second mask with green only
        mymask_2 = gray_2 < np.quantile(gray_2,quant) 
        mymask_2 = opening(mymask_2, struct_el)
        #third mask with greyscaled image
        mymask_3 = gray_3 < np.quantile(gray_3,quant) 
        mymask_3 = opening(mymask_3, struct_el)
        #Add all masks and keep joint elements
        better_mask = mymask_1.astype('int32')+mymask_2.astype('int32')+mymask_3.astype('int32')
        fixed_mask = better_mask.copy() + temp_seg

        #make mask binary (or rather 0's and 1's) then make morphology corrections
        fixed_mask[fixed_mask < 4] = 0
        fixed_mask[fixed_mask == 4] = 1
        fixed_mask = morphology.closing(fixed_mask,morphology.disk(3))
        fixed_mask = morphology.dilation(fixed_mask,morphology.disk(2))
        fixed_mask = morphology.area_closing(fixed_mask,100000)
        fixed_mask = morphology.remove_small_objects(fixed_mask.astype("bool"),3000)

        #calculate difference between current mask and original segmentation
        diff = np.abs((np.sum(fixed_mask))/np.sum(temp_seg)) 

        #keep the fixed_mask if it is at least 5% different than the original segmentation
        #mask cannot take 100% of the image size (768x1024)
        if((diff > 1.05 or diff < 0.95) and np.sum(fixed_mask)!=786432) :
            imlist.append(fixed_mask)
            difflist.append(np.abs(1-diff))
            index+=1
            same_counter = 0
        else:
            same_counter += 1


        #break when best mask has been found
        if index >=2:
            if difflist[index-1] > difflist[index-2] or same_counter >= 3:
                print(quant)
                break
        

            
    #print image
    mask = imlist[difflist.index(min(difflist))] if imlist else temp_seg
    
    new_filename = filename[:-4]+"_mask.jpg"                
    io.imsave('../data/interim/custom_masks/'+new_filename, img_as_ubyte(mask), check_contrast = False)

In [ ]:
problems = [images[23],images[24],images[30],images[31],images[40],images[50]
            ,images[52],images[77],images[82],images[94],images[108],images[114]
            ,images[121],images[123],images[124],images[127],images[132],images[136],images[141]]

In [ ]:
len(problems)